In [11]:
import pandas as pd 
import os 
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import requests

In [12]:
def list_of_towns():
    # silence
    options = webdriver.ChromeOptions()
    options.add_argument("headless")

    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver3', options=options)

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(5) # seconds

    # url of the county
    url = f"https://library.municode.com/VA"
    # headers to let them know who i am
    headers = {'user-agent': 'class project (hab6xf@virginia.edu)'}
    # xpath of the table in the webpage created by javascript 
    xpathHOME = "/html/body/div[1]/div[2]/ui-view/div[2]/section/div/div"

    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpathHOME)
    # links = driver.find_elements_by_tag_name("a")
    
    # add a delay of 3 seconds in the function
    time.sleep(2)
    
    # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
    html = data[0].get_attribute("outerHTML")

    r1 = re.findall(r"https://library.municode.com/VA/[\w\.-]+",html)

    # convert to dataframe and drop duplicates
    towns_list = pd.DataFrame(r1)
    towns_list = towns_list.drop_duplicates().reset_index(drop=True)

    return towns_list

In [13]:
town_urls = list_of_towns()
town_urls = town_urls.rename(columns={0:"urls"})
town_urls

,urls
0,https://library.municode.com/VA/accomack_county
1,https://library.municode.com/VA/albemarle_county
2,https://library.municode.com/VA/alexandria
3,https://library.municode.com/VA/alleghany_county
4,https://library.municode.com/VA/altavista
...,...
145,https://library.municode.com/VA/windsor
146,https://library.municode.com/VA/wise_county
147,https://library.municode.com/VA/woodstock
148,https://library.municode.com/VA/wytheville


In [14]:
town_urls.loc[0,'urls']

'https://library.municode.com/VA/accomack_county'

In [15]:
splits = town_urls['urls'].str.split("/")#[-1]

In [16]:
for x in range(len(splits)):
    town_urls.loc[x,'county'] = splits[x][-1]

In [17]:
town_urls

,urls,county
0,https://library.municode.com/VA/accomack_county,accomack_county
1,https://library.municode.com/VA/albemarle_county,albemarle_county
2,https://library.municode.com/VA/alexandria,alexandria
3,https://library.municode.com/VA/alleghany_county,alleghany_county
4,https://library.municode.com/VA/altavista,altavista
...,...,...
145,https://library.municode.com/VA/windsor,windsor
146,https://library.municode.com/VA/wise_county,wise_county
147,https://library.municode.com/VA/woodstock,woodstock
148,https://library.municode.com/VA/wytheville,wytheville


In [18]:
def identify_comparison_table_URL_part1(url):# 'https://library.municode.com/VA/accomack_county'):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver3', options=options)

    # set implicit wait time so that apis/javascript load before we scrape
    driver.implicitly_wait(5)  # seconds

    driver.get(url)
    driver.get_screenshot_as_file("headless.png")
    
    
    try:
        ref_table = driver.find_element_by_xpath(xpath="//*[@id='genToc_STLARETA']/a[2]")
        url = ref_table.get_attribute("href")
        return url
    except:
        pass
    try:
        ref_table = driver.find_element_by_xpath(xpath="//*[@id='genToc_STRETA']/a[2]")
        url = ref_table.get_attribute("href")
        return url
    except:
        pass
    try:
        ref_table = driver.find_element_by_xpath(xpath="//*[@id='genToc_STLARETA']/a[2]")
        url = ref_table.get_attribute("href")
        return url
    except:
        pass
    try:
        ref_table = driver.find_element_by_xpath(xpath="//*[@id='genToc_STATE_LAW_REFERENCE_TABLE']/a[2]")
        url = ref_table.get_attribute("href")
        return url
    except:
        pass
    try:
        ref_table = driver.find_element_by_xpath(xpath="//*[@id='genToc_COOR_STRETA']/a[2]")
        url = ref_table.get_attribute("href")
        return url
    except:
        pass
    # if nothing is returned, click open the genToc_THCH dropdown to expose more of the webpage 
#     try:
#         url = identify_comparison_table_URL_part1(url+"/codes/code_of_ordinances?nodeId=THCH")
# #         driver.get(url+"/codes/code_of_ordinances?nodeId=THCH")
# #         driver.get_screenshot_as_file("headless.png")
        
# # #         clickin = driver.find_element_by_xpath(xpath="//*[@id='genToc_THCH']/a[2]")
# # #         driver.implicitly_wait(5)
# #         ref_table = driver.find_element_by_xpath(xpath="//*[@id='genToc_THCH']/a[2]")
# #         url = ref_table.get_attribute("href")
#         return url
#     except:
#         pass

In [ ]:
ex = town_urls.loc[0,'urls']
ex

In [ ]:
identify_comparison_table_URL_part1(ex)

In [19]:
def version(url):

    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver3', options=options)

    # set implicit wait time so that apis/javascript load before we scrape
    driver.implicitly_wait(5)  # seconds

    driver.get(url)
    driver.get_screenshot_as_file("headless.png")
    
    try:
        version = driver.find_element_by_xpath(xpath="//*[@id='codebankToggle']/button[text()]")
        #url = version.get_attribute()
        #print("version:",version)
        #print("version:",version.text)
        version = version.text
        
        version = version.split(":")[-1].strip()#.split("(")[0].strip()
        

    except:
        print("couldn't get the version text")
        version = ''
        pass
    
    
    
    return version

In [ ]:
v = version(ex)
v

In [20]:
def scraper(url,town):
    # silence 
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")


    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver3', options=options)

    
    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(5) # seconds

    print("URL passed in",url)
    # url of the county
    try:
        url = identify_comparison_table_URL_part1(url)
        print("URL we're targeting:",url)
    
        # define the nodeID by taking the last piece of the ULR after the "="
        nodeID = url.split("=")[-1]

        # define the nodeIDs that we care about (discovered by team through manual check of Municode)
        nodeIDs = [
            'STLARETA',
            'STATE_LAW_REFERENCE_TABLE',
            'COOR_STRETA',
            'STRETA',
            'THCH_STRETA'
        ]

        # if nodeID not what we want, break
        if nodeID not in nodeIDs:
            print(f"pass: {town} empty")
            # define an empty dataframe
            df_empty = pd.DataFrame({'town' : []})

            # now write that empty dataframe to CSV
            df.to_csv(f'countyCSV/{town}.csv', index=False)  
            pass

        # if nodeID what we want, do this
        else:
            if nodeID == 'STLARETA':
                # xpath of the table in the webpage created by javascript 
                xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div/div/div[2]/table"
    # #                     "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section[2]/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div/div/div[2]
            elif nodeID == 'STRETA':
                xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section[2]/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div[2]/div/div[2]/table"

            elif nodeID == 'STATE_LAW_REFERENCE_TABLE':
                xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section[2]/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div/div/div[2]/table"

            elif nodeID == 'COOR_STRETA':
                xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section[2]/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div/div/div[2]/table"

            elif nodeID == 'THCH_STRETA':
                xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section[2]/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div/div/div[2]/table"

            driver.get(url)

            # use xpath to get to the table
            data = driver.find_elements_by_xpath(xpath)
            #print("data:",data)
            #if 

            # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
            table = data[0].get_attribute("outerHTML")
            #print("table:",table)

            # https://stackoverflow.com/questions/41214702/parse-html-and-read-html-table-with-selenium-python

            # convert that table into a pandas dataframe
            df = pd.read_html(table)
            df = df[0]


            # rename the columns
            df = df.rename(columns={'Code of Virginia  Section': "Virginia", "Section this Code":town})

            #print(url)

            # now write to CSV
            df.to_csv(f'countyCSV_March7/{town}.csv', index=False)  

            return df
    except:
        print(f"could not scrape for {town}")
        pass

# STLARETA

In [ ]:
scraper('https://library.municode.com/va/accomack_county',"accomack_county")

## STRETA

In [35]:
# https://library.municode.com/va/amherst_county/codes/code_of_ordinances?nodeId=STRETA

scraper('https://library.municode.com/va/amherst_county',"amherst_county")

URL passed in https://library.municode.com/va/amherst_county
URL we're targeting: https://library.municode.com/va/amherst_county/codes/code_of_ordinances?nodeId=STRETA


,0,1
0,Code of Virginia Section,Section this Code
1,1-7,1-7
2,1-9,1-8
3,1-10 et seq.,1-7
4,1-12,1-8
...,...,...
524,62.1-44.19:3.3,13-104
525,62.1-44.19:13(C),"App. A, § 917"
526,62.1-44.34:8,11.5-39
527,63.1-196,"App. A, § 302"


# STATE_LAW_REFERENCE_TABLE

In [36]:
# https://library.municode.com/va/bedford/codes/code_of_ordinances?nodeId=STATE_LAW_REFERENCE_TABLE

scraper('https://library.municode.com/va/bedford',"bedford")

URL passed in https://library.municode.com/va/bedford
URL we're targeting: https://library.municode.com/va/bedford/codes/code_of_ordinances?nodeId=STATE_LAW_REFERENCE_TABLE


,0,1
0,Code of Virginia Section,Section this Code
1,1-13.1,1-2
2,1-13.1—1-15,1-2
3,1-13.3,1-2
4,"1-13.6, 1-13.7",1-2
...,...,...
547,62.1-44.2 et seq.,"Ch. 58, Art. III"
548,63.1-175,"CHLDC, § 302"
549,NaN,"CHLDC, § 604"
550,63.1-196,"CHLDC, § 302"


# COOR_STRETA

In [41]:
scraper('https://library.municode.com/va/colonial_beach',"colonial_beach")

URL passed in https://library.municode.com/va/colonial_beach
URL we're targeting: https://library.municode.com/va/colonial_beach/codes/code_of_ordinances?nodeId=COOR_STRETA


,Virginia,Disposition
0,1-13.1,1-2
1,1-13.9,1-3
2,2.1-114.5:1,"Ch. 2, Art. III"
3,2.2-614.1,20-6
4,2.2-3700 et seq.,2-105
...,...,...
393,"59.1-142, 59.1-143 10-7","59.1-142, 59.1-143 10-7"
394,59.1-144,10-7
395,NaN,10-8
396,59.1-145—59.1-148 10-7,59.1-145—59.1-148 10-7


# THCH_STRETA

In [42]:
# https://library.municode.com/va/berryville/codes/code_of_ordinances?nodeId=THCH_STRETA

scraper('https://library.municode.com/va/berryville',"berryville")

URL passed in https://library.municode.com/va/berryville
URL we're targeting: None


AttributeError: 'NoneType' object has no attribute 'split'

# Run it

In [22]:
flagDF = pd.DataFrame()

for ind in town_urls.index:
    url = town_urls.loc[ind,'urls']
    print(url)
    town = town_urls.loc[ind,'county']
    print(town)
    
    flagDF.loc[ind,'locality'] = town
    flagDF.loc[ind,'version'] = version(url)
    
#     try:
#         flagDF.loc[ind,'locality'] = town
#         flagDF.loc[ind,'version'] = version(url)
        
#     except:
#         print(f'This municipality {town} does not host the necessary documents online.')
#         pass
    
    print('\n')
    
flagDF.to_csv('flagDF.csv', index=False) 

flagDF

https://library.municode.com/VA/accomack_county
accomack_county


https://library.municode.com/VA/albemarle_county
albemarle_county


https://library.municode.com/VA/alexandria
alexandria
couldn't get the version text


https://library.municode.com/VA/alleghany_county
alleghany_county


https://library.municode.com/VA/altavista
altavista


https://library.municode.com/VA/amherst
amherst


https://library.municode.com/VA/amherst_county
amherst_county


https://library.municode.com/VA/ashland
ashland
couldn't get the version text


https://library.municode.com/VA/bedford
bedford


https://library.municode.com/VA/bedford_county
bedford_county


https://library.municode.com/VA/berryville
berryville


https://library.municode.com/VA/big_stone_gap
big_stone_gap


https://library.municode.com/VA/blacksburg
blacksburg


https://library.municode.com/VA/blackstone
blackstone


https://library.municode.com/VA/bland_county
bland_county
couldn't get the version text


https://library.municode.com/V

,locality,version
0,accomack_county,"JAN 12, 2022 (CURRENT)"
1,albemarle_county,"MAR 22, 2022 (CURRENT)"
2,alexandria,
3,alleghany_county,"FEB 21, 2022 (CURRENT)"
4,altavista,"NOV 15, 2021 (CURRENT)"
...,...,...
145,windsor,"SEP 21, 2021 (CURRENT)"
146,wise_county,"OCT 23, 2019 (CURRENT)"
147,woodstock,"DEC 16, 2019 (CURRENT)"
148,wytheville,


In [13]:
import time
start_time = time.time()


flagDF = pd.DataFrame()

# loop through all of the towns 
# for ind in list_of_towns.index:
for ind in town_urls.index:
    url = town_urls.loc[ind,'urls']
    town = town_urls.loc[ind,'county']
    
    try:
        print(scraper(url,town))
    except:
        print(f'This municipality {town} does not host the necessary documents online.')
        pass
        
    print("\n")
        
print("--- %s seconds ---" % (time.time() - start_time))

URL passed in https://library.municode.com/VA/accomack_county
URL we're targeting: https://library.municode.com/va/accomack_county/codes/code_of_ordinances?nodeId=STLARETA
                             0                  1
0    Code of Virginia  Section  Section this Code
1                 1-13 et seq.              Ch. 1
2                       1-13.3                1-2
3               1-13.6, 1-13.7                1-2
4                       1-13.9                1-4
..                         ...                ...
401              62.1-44.15:46              38-98
402           62.1-148 et seq.   Ch. 102, Art. II
403                   62.1-255            106-376
404                   63.1-164              22-88
405          68.1-1200 et seq.              22-88

[406 rows x 2 columns]


URL passed in https://library.municode.com/VA/albemarle_county
URL we're targeting: None
could not scrape for albemarle_county
None


URL passed in https://library.municode.com/VA/alexandria
URL we're t